# COMP0189: Applied Artificial Intelligence
## Week 2 (pipelines and model selection)

### After this week you will be able to ...
- Streamline preprocessing steps in advanced way (Pipeline and ColumnTransformer)
- Perform model selection and model assessment using different partitions of the data
- Use different cross-validation strategies 
- Use GridSearchCV 

### Acknowledgements
- https://archive.ics.uci.edu/ml/datasets/adult
- https://scikit-learn.org/stable/modules/compose.html
- https://scikit-learn.org/stable/model_selection.html

## Preparation

Before we begin, we make sure that all necessary libraries are installed.

In [ ]:
%pip install scikit-learn==1.6.1 matplotlib seaborn pandas

In [2]:
from __future__ import annotations

We can now load the same adult income dataset used in last week's lab.

In [ ]:
import pandas as pd
import numpy as np
import numpy.typing as npt
from sklearn.model_selection import train_test_split

def load_adult(data_path: str) -> pd.DataFrame:
    # Load dataset with specifying ' ?' as missing values
    names = ['age','workclass','fnlwgt','education',
            'marital-status','occupation','relationship','race','sex',
            'capital-gain','capital-loss','hours-per-week','native-country',
            'y']
    df = pd.read_csv(data_path, delimiter=',', names=names, na_values=' ?')

    for col_name in df.columns:
        if df[col_name].dtype == object:  # Checking for object type (string in pandas)
            df[col_name] = df[col_name].apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    return df

def train_test_split_df(df: pd.DataFrame, test_ratio=0.3, target_col="y", random_state=42) -> tuple[pd.DataFrame, npt.NDArray[np.bool_], pd.DataFrame, npt.NDArray[np.bool_]]:
    # Separate features and target
    df_data = df.drop(columns=[target_col])
    df_target = df[target_col]

    # Randomized train-test split with a fixed seed
    train_X_df, test_X_df, train_y_df, test_y_df = train_test_split(
        df_data, df_target, test_size=test_ratio, random_state=random_state
    )

    # Convert target variable to binary (assuming it's categorical with ">50K" and others)
    train_y = (train_y_df == ">50K").to_numpy(dtype=np.bool_)
    test_y = (test_y_df == ">50K").to_numpy(dtype=np.bool_)

    return train_X_df, train_y, test_X_df, test_y

adult_df = load_adult("adult.data")
train_X, train_y, test_X, test_y = train_test_split_df(adult_df)

train_X

## Part 1: Pipelines

This part of the lab focuses on streamlining preprocessing and model training with scikit-learn. We will reimplement the last exercise of last week's lab using a more advanced way of controlling how to process data.

### Task 1.1: Writing a Pipeline

[Pipelines](https://scikit-learn.org/stable/modules/compose.html) are a feature of scikit-learn for combining multiple processing steps together. This is quite useful, for example, when we want to apply a sequence of many pre-processing steps to the same data.

In this exercise, we extract the `age` feature from the dataset and add some missing values for the sake of exercise. Use a `Pipeline` to impute the missing values and scale the feature.

> You can use the same imputation and scaling operations seen in the previous lab

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Function for visualising preprocessing results
def check_results(feature_name: str, age_original: npt.NDArray, age_preprocessed: npt.NDArray) -> None:
    # Plot the data distribution before and after preprocessing
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(feature_name)
    ax[0].hist(age_original)
    ax[0].set_title("Original")
    ax[1].hist(age_preprocessed)
    ax[1].set_title("Preprocessed")
    plt.show()

    # Print the value of the first entry before and after preprocessing
    print(f"Original first entry: {age_original.ravel()[0]}")
    print(f"Preprocessed first entry: {round(age_preprocessed.ravel()[0], 2)}")

# Set the first row of all columns to NaN to simulate missing values
train_X.iloc[0] = np.nan
test_X.iloc[0] = np.nan

# Extract the age feature as a Numpy array
age_original = train_X["age"].to_numpy(dtype=np.float32).reshape((len(train_X["age"]), 1))

# Create a Pipeline to preprocess the age feature
impute_and_scale = 

# Fit the pipeline and check results
age_preprocessed = impute_and_scale.fit_transform(age_original)
check_results("Age", age_original, age_preprocessed)

Notice that we can now easily reapply the fitted preprocessing steps to the testing data:

In [ ]:
# Extract the age feature from the testing data
age_original_test = test_X["age"].to_numpy(dtype=np.float32).reshape((len(test_X["age"]), 1))

# Apply the fitted pipeline
age_preprocessed_test = impute_and_scale.transform(age_original_test)

check_results("Age", age_original_test, age_preprocessed_test)

### Task 1.2: Deciding which Pipeline to run

In the previous task, we only applied our pipeline to the `age` feature, which we extracted and passed to the fitting function. However, the same imputation and scaling strategy would be suitable for most numerical features in the dataset. By contrast, it would not make sense for categorical features (since they cannot be scaled and cannot be imputed by taking the mean). So we'd like to use the same pipeline for all numerical features, while leaving out categorical ones.

Scikit-learn offers a [ColumnTransformer](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data) feature for this purpose. For this exercise, write a ColumnTransformer which takes the whole training dataset, selects only the numerical features and applies the pipeline from the previous cell.

In [ ]:
from sklearn.compose import ColumnTransformer

# List the features to process
non_categorical_features = ["age", ...]

# Apply the pipeline using ColumnTransformer
preprocessor = 

# Fit the pipeline and check results
non_categorical_preprocessed = 

for i, feature in enumerate(non_categorical_features):
    check_results(feature, train_X[[feature]].to_numpy(), non_categorical_preprocessed[:, i])

Applying the fitted pipeline to the testing data is now even easier, as we don't need to manually extract the features we want to preprocess.

Use the pipeline to transform the testing data.

In [ ]:
non_categorical_preprocessed = 

for i, feature in enumerate(non_categorical_features):
    check_results(feature, test_X[[feature]].to_numpy(), non_categorical_preprocessed[:, i])

### Task 1.3: Preprocessing the whole dataset

Now expand the code you've written so far to process all features in the dataset. Apply whichever steps seem reasonable depending on the feature types.

> You can reuse the same steps you used in last week's notebook, but use Pipelines to join them

In [ ]:
# Define your pipelines here

# Assign different pipelines to different features

# Fit the combined preprocessing pipeline to the training data

### Task 1.4: Joining preprocessing and model training

Note that any estimator can be part of a Pipeline - even models. Try to create a new pipeline which combines the `preprocessing_pipeline` from the previous task with an SVM classifier. The resulting pipeline will be able to both fit the preprocessing pipeline and train the model in a single step.

> You can use `make_pipeline` to add steps to an existing pipeline

Now use the fitted pipeline to preprocess the test data and predict results in a single step.

## Part 2: Cross Validation (CV)

scikit-learn provides a nice visualisation of various cross validation methods.
This notebook focuses on different cross validation strategies and how to account for data structure during cross-validation.


Visit: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#visualizing-cross-validation-behavior-in-scikit-learn

![kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png)
![stra-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_003.png)
![group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_004.png)
![stra-group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_010.png)

In [11]:
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold,
    GridSearchCV,
)
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np


def perform_cv(X_train: pd.DataFrame, y_train: npt.NDArray, pipeline: Pipeline, cv_strategy) -> tuple[float, float]:
    ''' 
    Return the mean accuracy and its standard deviation
    when fitting the given pipeline across several cross-validation folds,
    according to the given CV strategy
    '''

    scores = cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=cv_strategy,
        n_jobs=-1,
        scoring='accuracy'
    )
    
    return scores.mean(), scores.std()

### Task 5: Cross-validation for hyperparameter optimisation
Now apply cross-validation to the train set for optimizing the models hyperparameters. After identifying the best hyperparameter, measure the performance on the test data by training the models on the training data using the optimal hyperparameter. Print the average cross-validation score, the best cross-validation score, the best hyperparameter and the test-score.

 - Is there a difference between the average cross-validation score, the best cross-validation score and the test-score?

Note: remember that the pre-processing steps, including data centering and scaling should be embedded in the CV.



In [ ]:
############## Step 1: Cross-Validation for Hyperparameter Tuning ##############
# Define hyperparameter search space
C_values = [0.1, 0.5, 1.0]  # Regularization parameter
kernel_values = ['linear', 'rbf']  # Kernel types
cv_strategy = KFold(n_splits=2)  # 2-fold cross-validation

# Try all combinations and store results
svc_cv_results = []
for C in C_values:
    for kernel in kernel_values:
        mean_score, std_score = 
        svc_cv_results.append((C, kernel, mean_score, std_score))

############## Step 2: Train Final Model with Best Parameters ##############
# Find best parameters based on CV results
best_svc_params = max(svc_cv_results, key=lambda x: x[2])
print(f"\nBest parameters: C={best_svc_params[0]}, kernel={best_svc_params[1]}")

# Create and train final model with best parameters


############## Step 3: Evaluate on Test Set ##############

#### Task 5-1: Plotting model performance
Plot the model performance (mean accuracy and SD) for different hyper-parameter values.
- How does the accuracy vary as function of the hyperparameter?

### Task 6: Group Stratified cross-validation
Repeat task 5 using groups stratified CV with k=5 and sex as the group variable. Centre and scale the data before training the models. Print the average cross-validation score, the best cross-validation score, the best hyperparameter and the test-score.

- Did the performances changes with the group stratified CV?


### Task 7: Using Scikit's GridSearchCV
So far, we manually calculated the accuracy score of each cross-validation fold using the `perform_cv` function. This allowed us to show how cross-validation works in detail, but in practice we'd probably use the utilities provided by Scikit instead. One such utility is [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), a Scikit class to automatically perform hyperparameter optimisation by calculating the CV score for a "grid" of hyperparameters.

Here, rewrite the hyperparameter optimisation code so that it uses GridSearchCV to search the hyperparameters grid and train the final model.

See also the [user guide](https://scikit-learn.org/stable/modules/grid_search.html) for hyperparameter optimisation with Scikit.

In [ ]:
############## Step 1: Define Model and Pipeline ##############
# Create a pipeline that combines preprocessing and SVC model
# This ensures preprocessing steps are properly included in cross-validation
model_svc = make_pipeline(preprocessor, SVC(random_state=42))


############## Step 2: Define Hyperparameter Search Space ##############
# Specify the parameters to search
# Note: 'svc__' prefix is required because SVC is part of a pipeline
param_grid = {
    'svc__C': [0.1, 1, 10],          # Regularization parameter
    'svc__kernel': ['linear', 'rbf']  # Kernel types
}


############## Step 3: Setup Cross-validation Strategy ##############
# Use StratifiedKFold to maintain the same ratio of samples for each class


############## Step 4: Configure Grid Search ##############
# Initialize GridSearchCV with all parameters


############## Step 5: Perform Grid Search and fit the final model ##############
# Fit the grid search to find best parameters
# This step performs cross-validation for all parameter combinations


############## Step 6: Make Predictions ##############
# Apply model to test set


############## Step 7: Print Results ##############

### Task 8: Nested cross-validation
Now implement a nested CV for optimize the models’ hyper-parameters and assessing the models’ performance (with k=5 for the inner and outer loop). The inner loop should optimize the models’ hyper-parameters and the outer loop should assess the models’ performance.